In [ ]:
##!pip install openpyxl

In [2]:
import time
from typing import List, Dict, Optional, Set
from urllib.parse import quote_plus

import requests
from bs4 import BeautifulSoup
import pandas as pd


# Cabeçalho básico para evitar bloqueio por user-agent vazio
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    )
}


#### Helper

In [3]:

def get_text_or_none(elem) -> Optional[str]:
    """Retorna o texto stripado ou None se elem for None."""
    return elem.get_text(strip=True) if elem else None


def safe_criteria(criteria_list, index: int) -> Optional[str]:
    """Retorna um critério da lista (ex: experiência, tipo de contrato) se existir."""
    try:
        return criteria_list[index].get_text(strip=True)
    except (IndexError, AttributeError):
        return None

#### Buscador IDs

In [4]:

def fetch_job_ids(
    title: str,
    location: str,
    num_pages: int,
    pause: float = 0.5,
    verbose: bool = True,
) -> List[str]:
    """
    Busca job_ids de vagas no LinkedIn Jobs (endpoint 'seeMoreJobPostings').

    :param title: termo de busca (ex: 'engineer', 'scientist').
    :param location: localização (ex: 'Brazil', 'Lisbon, Portugal').
    :param num_pages: número de páginas a percorrer.
    :param pause: tempo de espera entre requisições (segundos).
    :param verbose: se True, imprime logs simples.
    :return: lista de job_ids (sem duplicados).
    """
    job_ids: Set[str] = set()

    keywords = f"data {title}".strip()
    keywords_encoded = quote_plus(keywords)
    location_encoded = quote_plus(location)

    with requests.Session() as session:
        session.headers.update(HEADERS)

        for page in range(num_pages):
            start = page * 10  # cada página costuma ter 10 vagas
            list_url = (
                "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
                f"?keywords={keywords_encoded}"
                f"&location={location_encoded}"
                "&position=1&pageNum=0"
                f"&start={start}"
            )

            try:
                response = session.get(list_url, timeout=10)
            except requests.RequestException as e:
                if verbose:
                    print(f"[ERRO] Falha na requisição da página {page}: {e}")
                continue

            if verbose:
                print(f"[LISTA] Página {page} - status {response.status_code}")

            if response.status_code != 200:
                if verbose:
                    print(f"[AVISO] Pulando página {page} (status {response.status_code})")
                continue

            soup = BeautifulSoup(response.text, "html.parser")
            page_jobs = soup.find_all("li")

            if not page_jobs and verbose:
                print(f"[AVISO] Nenhuma vaga encontrada na página {page}")

            for job in page_jobs:
                base_card_div = job.find("div", {"class": "base-card"})
                if not base_card_div:
                    continue

                urn = base_card_div.get("data-entity-urn")
                if not urn:
                    continue

                parts = urn.split(":")
                if len(parts) < 4:
                    continue

                job_id = parts[3]
                job_ids.add(job_id)
                if verbose:
                    print(f"[JOB ID] {job_id}")

            time.sleep(pause)

    return list(job_ids)

#### Scrape de detalhes da vaga

In [5]:
def parse_job_page(html: str, job_id: str) -> Dict[str, Optional[str]]:
    """Extrai os dados de uma página de vaga do LinkedIn."""
    soup = BeautifulSoup(html, "html.parser")
    job_post: Dict[str, Optional[str]] = {}

    # Título da vaga
    job_post["job_title"] = get_text_or_none(
        soup.find(
            "h2",
            {
                "class": (
                    "top-card-layout__title font-sans text-lg "
                    "papabear:text-xl font-bold leading-open "
                    "text-color-text mb-0 topcard__title"
                )
            },
        )
    )

    # Nome da empresa
    job_post["company_name"] = get_text_or_none(
        soup.find(
            "a",
            {"class": "topcard__org-name-link topcard__flavor--black-link"},
        )
    )

    # Critérios (nível de experiência, tipo de contrato, etc.)
    criteria = soup.find_all(
        "span",
        {
            "class": (
                "description__job-criteria-text "
                "description__job-criteria-text--criteria"
            )
        },
    )

    job_post["experience_level"] = safe_criteria(criteria, 0)
    job_post["type_of_contract"] = safe_criteria(criteria, 1)
    # Se quiser, dá pra reativar:
    # job_post["job_role"] = safe_criteria(criteria, 2)
    # job_post["field"] = safe_criteria(criteria, 3)

    # Easy apply (True/False)
    try:
        offsite_icon = soup.find(
            "icon",
            {"data-svg-class-name": "apply-button__offsite-apply-icon-svg"},
        )
        job_post["easy_apply"] = False if offsite_icon else True
    except Exception:
        job_post["easy_apply"] = None

    # Tempo desde a postagem
    job_post["time_posted"] = get_text_or_none(
        soup.find(
            "span",
            {
                "class": (
                    "posted-time-ago__text "
                    "topcard__flavor--metadata"
                )
            },
        )
    )

    # Número de candidatos
    job_post["num_applicants"] = get_text_or_none(
        soup.find("span", {"class": "num-applicants__caption"})
    )

    # Link da vaga
    job_post["job_link"] = f"https://www.linkedin.com/jobs/view/{job_id}/"

    return job_post


def scrape_jobs(
    id_list: List[str],
    sleep_seconds: float = 0.5,
    verbose: bool = True,
) -> List[Dict[str, Optional[str]]]:
    """
    Faz scrape dos detalhes de uma lista de job_ids do LinkedIn.

    :param id_list: lista de IDs de vagas.
    :param sleep_seconds: pausa entre requisições, para evitar 429.
    :param verbose: se True, imprime status das requisições.
    :return: lista de dicionários com dados da vaga.
    """
    job_list: List[Dict[str, Optional[str]]] = []

    with requests.Session() as session:
        session.headers.update(HEADERS)

        for job_id in id_list:
            job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

            try:
                response = session.get(job_url, timeout=10)
            except requests.RequestException as e:
                if verbose:
                    print(f"[ERRO] Falha ao buscar job {job_id}: {e}")
                continue

            if verbose:
                print(f"[DETALHE] {job_id} - status {response.status_code}")

            if response.status_code != 200:
                if verbose:
                    print(f"[AVISO] Pulando job {job_id} (status {response.status_code})")
                continue

            job_post = parse_job_page(response.text, job_id)
            job_list.append(job_post)

            time.sleep(sleep_seconds)

    return job_list


In [6]:
def run_linkedin_pipeline(
    title: str,
    location: str,
    num_pages: int,
    out_csv: Optional[str] = None,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Roda o pipeline completo:
    - Busca job_ids
    - Scrapa detalhes
    - Retorna DataFrame e opcionalmente salva CSV
    """
    if verbose:
        print("=== Buscando job IDs ===")
    ids = fetch_job_ids(title, location, num_pages, verbose=verbose)

    if verbose:
        print(f"\nTotal de IDs encontrados: {len(ids)}")
        print("=== Scraping detalhes das vagas ===")

    job_list = scrape_jobs(ids, verbose=verbose)

    df = pd.DataFrame(job_list)

    if out_csv is not None:
        df.to_csv(out_csv, index=False, encoding="utf-8-sig")
        if verbose:
            print(f"\n[OK] Dados salvos em: {out_csv}")

    return df

In [7]:
import pandas as pd

def salvar_em_excel(
    df: pd.DataFrame,
    caminho_arquivo: str = "linkedin_jobs.xlsx",
    sheet_name: str = "Vagas",
    col_widths: dict | None = None,  # ex: {"job_title": 60, "company_name": 30}
    auto_row_height: bool = False,
):
    """
    Salva DataFrame em Excel com ajuste de largura das colunas.
    
    :param df: DataFrame com os dados.
    :param caminho_arquivo: nome/ caminho do arquivo .xlsx.
    :param sheet_name: nome da aba.
    :param col_widths: dict opcional {nome_coluna: largura_em_excel}.
    :param auto_row_height: se True, ativa quebra de linha (wrap_text),
                            o que ajuda em textos muito grandes.
    """
    from openpyxl.utils import get_column_letter
    from openpyxl.styles import Alignment

    if col_widths is None:
        col_widths = {}

    with pd.ExcelWriter(caminho_arquivo, engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name=sheet_name)

        ws = writer.sheets[sheet_name]

        # 1) Ajuste automático de largura baseado no conteúdo
        for col_idx, column in enumerate(ws.columns, start=1):
            max_length = 0
            col_letter = get_column_letter(col_idx)

            for cell in column:
                value = cell.value
                if value is not None:
                    value_len = len(str(value))
                    if value_len > max_length:
                        max_length = value_len

            # margem extra
            adjusted_width = max_length + 2
            ws.column_dimensions[col_letter].width = adjusted_width

        # 2) Sobrescrever larguras específicas via col_widths
        #    (usa nome da coluna do DataFrame)
        header_row = next(ws.iter_rows(min_row=1, max_row=1))
        col_name_to_letter = {
            cell.value: cell.column_letter for cell in header_row if cell.value
        }

        for col_name, width in col_widths.items():
            if col_name in col_name_to_letter:
                ws.column_dimensions[col_name_to_letter[col_name]].width = width

        # 3) (Opcional) Quebra de linha + altura de linha (para textos longos)
        if auto_row_height:
            for row in ws.iter_rows(min_row=1):
                for cell in row:
                    cell.alignment = Alignment(wrap_text=True)
            # Excel recalcula a altura automaticamente quando wrap_text está ativo

    print(f"Arquivo Excel salvo em: {caminho_arquivo}")


In [8]:
import openpyxl

df = run_linkedin_pipeline(
    title="engineer",
    location="Brazil",
    num_pages=3,
    verbose=True,
)

col_widths_personalizadas = {
    "job_title": 60,        # título bem largo
    "company_name": 30,     # empresa médio
    "experience_level": 20,
    "type_of_contract": 20,
    "num_applicants": 18,
    "easy_apply": 12,
    "time_posted": 18,
    "job_link": 50,         # URL grande
}

salvar_em_excel(
    df,
    caminho_arquivo="linkedin_jobs_brazil_engineer.xlsx",
    sheet_name="Vagas",
    col_widths=col_widths_personalizadas,
    auto_row_height=True,   # quebra linha em textos longos
)



=== Buscando job IDs ===
[LISTA] Página 0 - status 200
[JOB ID] 4318327757
[JOB ID] 4308497997
[JOB ID] 4281828542
[JOB ID] 4233957493
[JOB ID] 4337472760
[JOB ID] 4312919215
[JOB ID] 4298239221
[JOB ID] 4324753186
[JOB ID] 4310476364
[JOB ID] 4317491191
[LISTA] Página 1 - status 200
[JOB ID] 4303420983
[JOB ID] 4231033954
[JOB ID] 4312301348
[JOB ID] 4316823422
[JOB ID] 4331368566
[JOB ID] 4314223388
[JOB ID] 3969555543
[JOB ID] 4334041808
[JOB ID] 4315447335
[JOB ID] 4331348122
[LISTA] Página 2 - status 200
[JOB ID] 4334386540
[JOB ID] 4307252914
[JOB ID] 4245979421
[JOB ID] 4333019146
[JOB ID] 4320660148
[JOB ID] 4307685458
[JOB ID] 4214699583
[JOB ID] 4295464589
[JOB ID] 4313394952
[JOB ID] 4312861310

Total de IDs encontrados: 30
=== Scraping detalhes das vagas ===
[DETALHE] 4310476364 - status 200
[DETALHE] 4214699583 - status 200
[DETALHE] 4318327757 - status 200
[DETALHE] 4316823422 - status 200
[DETALHE] 4337472760 - status 200
[DETALHE] 4303420983 - status 200
[DETALHE] 39695